Import packages.

In [55]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

## The Backward Euler method

We analyze the semi-batch reactor. The reactor model contains a non-autonomous term (right hand-side contains the independent variable $t$ explicitly).

$\frac{dC_A}{dt}=\frac{\dot{V}}{V_0+\dot{V}t}\left(C_A^{in}-C_A\right)-kC_A$

$\frac{dC_B}{dt}=kC_A-\frac{\dot{V}}{V_0+\dot{V}t}C_B$

Initial contition:

$C_A(t=0\,s)=1\, mol/L,\; C_B(t=0\,s)=0\, mol/L$

Define parameters.

In [56]:
# time domain
t_start = 0.0
t_end = 30.0
i = 500
t = np.linspace(t_start, t_end, i)

# initial conditions
c0 = np.array([1,0])

Define chemical reaction system.

In [57]:
def dC(t: np.ndarray, c: np.ndarray) -> np.ndarray:
    """ODE system for semi-batch reactor.

    Parameters
    ----------
    t : np.ndarray
        Time variable
    c : np.ndarray
        Concentration of individual components

    Returns
    -------
    np.ndarray
        Concentration gradient in reactor
    """
    cA, cB = c
    k = 0.2
    Vdot = 0.1
    V_0 = 10
    cA_in = 0.5
    dcA = Vdot/(V_0+Vdot*t)*(cA_in-cA)-k*cA
    dcB = k*cA-Vdot/(V_0+Vdot*t)*cB
    return np.array([dcA, dcB])

Define backward euler method.

In [58]:
def backward_euler(func: Callable, y0: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Generic backward Euler method for initial value problem. Use scipy's 
    fsolve to solve root finding problem.

    Parameters
    ----------
    func : Callable
        Function that defines the ODE (y' = func(t, y)).
    y0 : np.ndarray
        Initial condition.
    t : np.ndarray
        Time domain.

    Returns
    -------
    np.ndarray
        Array of solution values at the time points.
    """
    # Initialize arrays for time and solution values
    y = np.zeros([len(y0),len(t)])
    y[:,0] = y0
    dt = t[1]-t[0]

    # Iterate over each time step
    for i in range(len(t)-1):
        # Initial guess for y_{i+1}
        y_guess = y[:,i]
        # Define backward Euler function
        euler = lambda y_next: y[:,i] + dt * func(t[i+1], y_next) - y_next

        # Update solution
        y[:,i+1] = scipy.optimize.fsolve(euler, y_guess)
    
    return y

Execute backward euler method.

In [59]:
c = backward_euler(dC, c0, t)

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, c[0,:], label = "cA")
ax.plot(t, c[1,:], label = "cB")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("The Backward Euler method")
ax.legend()
ax.grid()
fig.show()